Install needed libraries

In [1]:
%pip install requests
%pip install PyGithub
%pip install openai
%pip install --upgrade openai
%pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


Imports

In [2]:
from github import Github
from openai import OpenAI,AzureOpenAI
import json
import requests
import re
from dotenv import load_dotenv
import os

Function to check if input is already a GitHub repo

In [54]:
def is_github_repo_link(env):
    return env.startswith("https://github.com/") or env.startswith("http://github.com/")

Prompt for user

In [89]:
prompt = input("Enter a name of the environment or link to GitHub repo: ")
flag = True
if is_github_repo_link(prompt) == False:
   flag = False

Enter a name of the environment or link to GitHub repo: angular


Function for searching GitHub repositories

In [90]:
def retrieve_files(repo_urls):
    file_names = ['README.md', 'install.md', 'package.json']
    files_content = {file_name: [] for file_name in file_names}

    for url in repo_urls:
        if url.endswith('.git'):
            repo_name = url[:-4].split('github.com/')[-1]
        else:
            repo_name = url.split('github.com/')[-1]

        branches = ['main', 'master']
        for branch in branches:
            api_url = f"https://api.github.com/repos/{repo_name}/git/trees/{branch}?recursive=1"

            try:
                response = requests.get(api_url)
                if response.status_code == 200:
                    tree = response.json().get('tree', [])
                    for file_info in tree:
                        file_path = file_info['path']
                        file_name = file_path.split('/')[-1]
                        if file_name in file_names:
                            file_url = f"https://raw.githubusercontent.com/{repo_name}/{branch}/{file_path}"
                            file_response = requests.get(file_url)
                            if file_response.status_code == 200:
                                files_content[file_name].append(file_response.text)
                                print(f"Successfully retrieved {file_name} from {file_url}")
                            else:
                                print(f"Failed to retrieve {file_name} from {file_url}. Status code: {file_response.status_code}")
                    break  # Exit loop if successful
                elif response.status_code == 404:
                    print(f"Branch '{branch}' not found for repository {repo_name}.")
                else:
                    print(f"Failed to retrieve file list from {api_url}. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error while trying to get file list from {api_url}: {e}")

    return files_content

OpenAI API for getting repositories

In [91]:
load_dotenv()
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint="https://daytonaai-us.openai.azure.com/",
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)
def generate_github_repos(prompt, model="gpt-4o"):
    system_prompt = "You are an AI assistant that suggests relevant GitHub repositories based on a project description."
    user_prompt = f"Suggest GitHub repositories for a project that involves the following technologies: {prompt}. Provide only the URLs of the repositories."

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    suggested_repos = response.choices[0].message.content.strip()
    repo_urls = re.findall(r'(https?://github\.com/[^\s]+)', suggested_repos)
    return repo_urls
repo_urls = generate_github_repos(prompt)
if repo_urls:
    print(repo_urls)
else:
    print("Failed to fetch repository URLs.")

['https://github.com/angular/angular', 'https://github.com/ngx-translate/core', 'https://github.com/ngrx/platform', 'https://github.com/angular/angular-cli', 'https://github.com/angular/components']


Analyze and pre-process the data

In [81]:
def parse_readme(content):
    sections = {}
    current_section = None
    lines = content.split('\n')

    for line in lines:
        header_match = re.match(r'^(#+)\s+(.*)', line)
        if header_match:
            level = len(header_match.group(1))
            section_name = header_match.group(2).strip()
            current_section = section_name
            sections[current_section] = []
        elif current_section:
            sections[current_section].append(line)

    for section in sections:
        sections[section] = "\n".join(sections[section])

    return sections

In [75]:
def parse_package_json(content):
    try:
        package_data = json.loads(content)
        return {
            "dependencies": package_data.get("dependencies", {}),
            "devDependencies": package_data.get("devDependencies", {}),
            "scripts": package_data.get("scripts", {})
        }
    except json.JSONDecodeError as e:
        print(f"Error parsing package.json: {e}")
        return {}

In [82]:
def collect_context(files_content):
    context = {}

    for file_name, contents in files_content.items():
        if file_name == 'README.md':
            for content in contents:
                context[file_name] = parse_readme(content)
        elif file_name == 'package.json':
            for content in contents:
                context[file_name] = parse_package_json(content)

    return context

Generate JSON file for Dev Container with OpenAI API

In [87]:
load_dotenv()
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint="https://daytonaai-us.openai.azure.com/",
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)
def generate_dev_container_json(prompt, model="gpt-4o"):
    system_prompt = "You are an AI assistant that creates JSON file for dev container.\n\n"
    user_prompt = f"Generate JSON configuration for dev container based on the context provided: {context}"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )


    print("Raw API Response:")
    print(response)

    if 'data' in response:
        for message in response['data']:
            if message['role'] == 'assistant':
                generated_json = message['content']
                try:
                    parsed_json = json.loads(generated_json)
                    print("Generated Dev Container JSON Configuration:")
                    print(json.dumps(parsed_json, indent=4))
                    return parsed_json
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                    return None
            else:
                print(f"Role '{message['role']}' not recognized.")
                return None
    else:
        print("No 'data' key found in API response.")
        return None


In [92]:
if flag == False:
  repos=generate_github_repos(prompt)
  files=retrieve_files(repos)
else:
  repo=[prompt]
  files=retrieve_files(repo)
context = collect_context(files)
generate_dev_container_json(context)

Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.circleci/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.devcontainer/README.md
Successfully retrieved package.json from https://raw.githubusercontent.com/angular/angular/main/.github/actions/deploy-docs-site/package.json
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.vscode/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.yarn/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/adev/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/adev/scripts/update-cli-help/README.md
Successfully retrieved README.md from https://raw.githubuserconte